# <div align="center">Python Classes</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

In Python, everything is an object – everything is an instance of some class. In earlier versions of Python a distinction was made between built-in types and user-defined classes, but these are now completely indistinguishable. Classes and types are themselves objects, and they are of type type. You can find out the type of any object using the type function:

In [2]:
type(any_object)

Here is an example of a simple custom class which stores information about a person:

In [3]:
import datetime # we will use this for date objects

class Person:

    def __init__(self, name, surname, birthdate, address, telephone, email):
        self.name = name
        self.surname = surname
        self.birthdate = birthdate

        self.address = address
        self.telephone = telephone
        self.email = email

    def age(self):
        today = datetime.date.today()
        age = today.year - self.birthdate.year

        if today < datetime.date(today.year, self.birthdate.month, self.birthdate.day):
            age -= 1

        return age

person = Person(
    "Jane",
    "Doe",
    datetime.date(1992, 3, 12), # year, month, day
    "No. 12 Short Street, Greenville",
    "555 456 0987",
    "jane.doe@example.com"
)

print(person.name)
print(person.email)
print(person.age())

Jane
jane.doe@example.com
27


We start the class definition with the class keyword, followed by the class name and a colon. We would list any parent classes in between round brackets before the colon, but this class doesn’t have any, so we can leave them out.

Inside the class body, we define two functions – these are our object’s methods. The first is called \__init__, which is a special method. When we call the class object, a new instance of the class is created, and the \__init__ method on this new object is immediately executed with all the parameters that we passed to the class object. The purpose of this method is thus to set up a new object using data that we have provided.

The second method is a custom method which calculates the age of our person using the birthdate and the current date.

\__init__ is sometimes called the object’s constructor, because it is used similarly to the way that constructors are used in other languages, but that is not technically correct – it’s better to call it the initialiser. There is a different method called \__new__ which is more analogous to a constructor, but it is hardly ever used.

ou may have noticed that both of these method definitions have self as the first parameter, and we use this variable inside the method bodies – but we don’t appear to pass this parameter in. This is because whenever we call a method on an object, the object itself is automatically passed in as the first parameter. This gives us a way to access the object’s properties from inside the object’s methods.

In some languages this parameter is implicit – that is, it is not visible in the function signature – and we access it with a special keyword. In Python it is explicitly exposed. It doesn’t have to be called self, but this is a very strongly followed convention.



# <div align="center">Instance attributes</div>
--------------------------------------------------------------------

It is important to note that the attributes set on the object in the __init__ function do not form an exhaustive list of all the attributes that our object is ever allowed to have.

In some languages you must provide a list of the object’s attributes in the class definition, placeholders are created for these allowed attributes when the object is created, and you may not add new attributes to the object later. In Python, you can add new attributes, and even new methods, to an object on the fly. In fact, there is nothing special about the __init__ function when it comes to setting attributes. We could store a cached age value on the object from inside the age function:



In [4]:
def age(self):
    if hasattr(self, "_age"):
        return self._age

    today = datetime.date.today()

    age = today.year - self.birthdate.year

    if today < datetime.date(today.year, self.birthdate.month, self.birthdate.day):
        age -= 1

    self._age = age
    return age

Starting an attribute or method name with an underscore (_) is a convention which we use to indicate that it is a “private” internal property and should not be accessed directly. In a more realistic example, our cached value would sometimes expire and need to be recalculated – so we should always use the age method to make sure that we get the right value.

# <div align="center">Class attributes</div>
--------------------------------------------------------------------

All the attributes which are defined on a Person instance are instance attributes – they are added to the instance when the \__init__ method is executed. We can, however, also define attributes which are set on the class. These attributes will be shared by all instances of that class. In many ways they behave just like instance attributes, but there are some caveats that you should be aware of.

We define class attributes in the body of a class, at the same indentation level as method definitions (one level up from the insides of methods):

In [4]:
class Person:

    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, title, name, surname):
        if title not in self.TITLES:
            raise ValueError("%s is not a valid title." % title)

        self.title = title
        self.name = name
        self.surname = surname

As you can see, we access the class attribute TITLES just like we would access an instance attribute – it is made available as a property on the instance object, which we access inside the method through the self variable.

All the Person objects we create will share the same TITLES class attribute.

Class attributes are often used to define constants which are closely associated with a particular class. Although we can use class attributes from class instances, we can also use them from class objects, without creating an instance:

In [7]:
# we can access a class attribute from an instance
person.TITLES

# but we can also access it from the class
Person.TITLES

Note that the class object doesn’t have access to any instance attributes – those are only created when an instance is created!

When we set an attribute on an instance which has the same name as a class attribute, we are ***overriding*** the class attribute with an instance attribute, which will take precedence over it. If we create two Person objects and call the mark_as_deceased method on one of them, we will not affect the other one. We should, however, be careful when a class attribute is of a mutable type – because if we modify it in-place, we will affect all objects of that class at the same time. Remember that all instances share the same class attributes:

In [8]:
class Person:
    pets = []

    def add_pet(self, pet):
        self.pets.append(pet)

jane = Person()
bob = Person()

jane.add_pet("cat")
print(jane.pets)
print(bob.pets) # oops!

What we should do in cases like this is initialise the mutable attribute as an instance attribute, inside \__init__. Then every instance will have its own separate copy:

In [9]:
class Person:

    def __init__(self):
        self.pets = []

    def add_pet(self, pet):
        self.pets.append(pet)

jane = Person()
bob = Person()

jane.add_pet("cat")
print(jane.pets)
print(bob.pets)

['cat']
[]


Note that method definitions are in the same scope as class attribute definitions, so we can use class attribute names as variables in method definitions (without self, which is only defined inside the methods):

In [10]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, title, name, surname, allowed_titles=TITLES):
        if title not in allowed_titles:
            raise ValueError("%s is not a valid title." % title)

        self.title = title
        self.name = name
        self.surname = surname

Can we have class methods? Yes, we can. In the next section we will see how to define them using a decorator.

# <div align="center">Class decorators</div>
--------------------------------------------------------------------

There are some built-in decorators which are often used in class definitions.

## @classmethod

Just like we can define class attributes, which are shared between all instances of a class, we can define class methods. We do this by using the @classmethod decorator to decorate an ordinary method.

A class method still has its calling object as the first parameter, but ***by convention we rename this parameter from self to cls***. If we call the class method from an instance, this parameter will contain the instance object, but if we call it from the class it will contain the class object. By calling the parameter cls we remind ourselves that it is not guaranteed to have any instance attributes.

What are class methods good for? Sometimes there are tasks associated with a class which we can perform using constants and other class attributes, without needing to create any class instances. If we had to use instance methods for these tasks, we would need to create an instance for no reason, which would be wasteful. Sometimes we write classes purely to group related constants together with functions which act on them – we may never instantiate these classes at all.

Sometimes it is useful to write a class method which creates an instance of the class after processing the input so that it is in the right format to be passed to the class constructor. This allows the constructor to be straightforward and not have to implement any complicated parsing or clean-up code:

In [11]:
class Person:

    def __init__(self, name, surname, birthdate, address, telephone, email):
        self.name = name
        # (...)

    @classmethod
    def from_text_file(cls, filename):
        # extract all the parameters from the text file
        return cls(*params) # this is the same as calling Person(*params)

## @staticmethod

A static method doesn’t have the calling object passed into it as the first parameter. This means that it doesn’t have access to the rest of the class or instance at all. We can call them from an instance or a class object, but they are ***most commonly called from class objects, like class methods***.

If we are using a class to group together related methods which don’t need to access each other or any other data on the class, we may want to use this technique. The advantage of using static methods is that we eliminate unnecessary cls or self parameters from our method definitions. The disadvantage is that if we do occasionally want to refer to another class method or attribute inside a static method we have to write the class name out in full, which can be much more verbose than using the cls variable which is available to us inside a class method.

Here is a brief example comparing the three method types:

In [12]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self): # instance method
        # instance object accessible through self
        return "%s %s" % (self.name, self.surname)

    @classmethod
    def allowed_titles_starting_with(cls, startswith): # class method
        # class or instance object accessible through cls
        return [t for t in cls.TITLES if t.startswith(startswith)]

    @staticmethod
    def allowed_titles_ending_with(endswith): # static method
        # no parameter for class or instance object
        # we have to use Person directly
        return [t for t in Person.TITLES if t.endswith(endswith)]


jane = Person("Jane", "Smith")

print(jane.fullname())

print(jane.allowed_titles_starting_with("M"))
print(Person.allowed_titles_starting_with("M"))

print(jane.allowed_titles_ending_with("s"))
print(Person.allowed_titles_ending_with("s"))

Jane Smith
['Mr', 'Mrs', 'Ms']
['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']
['Mrs', 'Ms']


## @property

Sometimes we use a method to generate a property of an object dynamically, calculating it from the object’s other properties. Sometimes you can simply use a method to access a single attribute and return it. You can also use a different method to update the value of the attribute instead of accessing it directly. Methods like this are called ***getters and setters***, because they “get” and “set” the values of attributes, respectively.

In some languages you are encouraged to use getters and setters for all attributes, and never to access their values directly – and there are language features which can make attributes inaccessible except through setters and getters. In Python, accessing simple attributes directly is perfectly acceptable, and writing getters and setters for all of them is considered unnecessarily verbose. Setters can be inconvenient because they don’t allow use of compound assignment operators:

In [13]:
class Person:
    def __init__(self, height):
        self.height = height

    def get_height(self):
        return self.height

    def set_height(self, height):
        self.height = height

jane = Person(153) # Jane is 153cm tall

jane.height += 1 # Jane grows by a centimetre
jane.set_height(jane.height + 1) # Jane grows again

As we can see, incrementing the height attribute through a setter is much more verbose. Of course we could write a second setter which increments the attribute by the given parameter – but we would have to do something similar for every attribute and every kind of modification that we want to perform. We would have a similar issue with in-place modifications, like adding values to lists.

Something which is often considered an advantage of setters and getters is that we can change the way that an attribute is generated inside the object without affecting any code which uses the object. For example, suppose that we initially created a Person class which has a fullname attribute, but later we want to change the class to have separate name and surname attributes which we combine to create a full name. If we always access the fullname attribute through a setter, we can just rewrite the setter – none of the code which calls the setter will have to be changed.

But what if our code accesses the fullname attribute directly? We can write a fullname method which returns the right value, but a method has to be called. Fortunately, the ***@property decorator lets us make a method behave like an attribute***:

In [14]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def fullname(self):
        return "%s %s" % (self.name, self.surname)

jane = Person("Jane", "Smith")
print(jane.fullname) # no brackets!

Jane Smith


There are also decorators which we can use to define a setter and a deleter for our attribute (a deleter will delete the attribute from our object). ***The getter, setter and deleter methods must all have the same name***:

In [15]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def fullname(self):
        return "%s %s" % (self.name, self.surname)

    @fullname.setter
    def fullname(self, value):
        # this is much more complicated in real life
        name, surname = value.split(" ", 1)
        self.name = name
        self.surname = surname

    @fullname.deleter
    def fullname(self):
        del self.name
        del self.surname

jane = Person("Jane", "Smith")
print(jane.fullname)

jane.fullname = "Jane Doe"
print(jane.fullname)
print(jane.name)
print(jane.surname)

Jane Smith
Jane Doe
Jane
Doe


# <div align="center">Inspecting an object</div>
--------------------------------------------------------------------

We can check what properties are defined on an object using the dir function:

In [16]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self):
        return "%s %s" % (self.name, self.surname)

jane = Person("Jane", "Smith")

print(dir(jane))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fullname', 'name', 'surname']


Now we can see our attributes and our method – but what’s all that other stuff? We will discuss inheritance in the next chapter, but for now all you need to know is that any class that you define has object as its parent class even if you don’t explicitly say so – so your class will have a lot of default attributes and methods that any Python object has.

***Note:*** in Python 2 we have to inherit from object explicitly, otherwise our class will be almost completely empty except for our own custom properties. Classes which don’t inherit from object are called “old-style classes”, and using them is not recommended. If we were to write the person class in Python 2 we would write the first line as class Person(object):.

This is why you can just leave out the \__init__ method out of your class if you don’t have any initialisation to do – the default that you inherited from object (which does nothing) will be used instead. If you do write your own \__init__ method, it will ***override the default method. Sometimes we also call this overloading.***

Many default methods and attributes that are found in built-in Python objects have names which begin and end in double underscores, like \__init__ or \__str__. These names indicate that these properties have a special meaning – you shouldn’t create your own methods or attributes with the same names unless you mean to overload them. These properties are usually methods, and they are sometimes called ***magic methods.***

Here are some examples of special object properties:

* \__init__: the initialisation method of an object, which is called when the object is created.


* \__str__: the string representation method of an object, which is called when you use the str function to convert that object to a string.


* \__class__: an attribute which stores the the class (or type) of an object – this is what is returned when you use the type function on the object.


* \__eq__: a method which determines whether this object is equal to another. There are also other methods for determining if it’s not equal, less than, etc.. These methods are used in object comparisons, for example when we use the equality operator == to check if two objects are equal.


* \__add__ is a method which allows this object to be added to another object. There are equivalent methods for all the other arithmetic operators. Not all objects support all arithemtic operations – numbers have all of these methods defined, but other objects may only have a subset.


* \__iter__: a method which returns an iterator over the object – we will find it on strings, lists and other iterables. It is executed when we use the iter function on the object.


* \__len__: a method which calculates the length of an object – we will find it on sequences. It is executed when we use the len function of an object.


* \__dict__: a dictionary which contains all the instance attributes of an object, with their names as keys. It can be useful if we want to iterate over all the attributes of an object. \__dict__  does not include any methods, class attributes or special default attributes like \__class__.

# <div align="center">Overriding magic methods</div>
--------------------------------------------------------------------

We have already seen how to overload the \__init\__ method so that we can customise it to initialise our class. We can also overload other special methods. For example, the purpose of the \__str\__ method is to output a useful string representation of our object. but by default if we use the str function on a person object (which will call the \__str\__ method), all that we will get is the class name and an ID. That’s not very useful! Let’s write a custom \__str\__ method which shows the values of all of the object’s properties:

In [17]:
import datetime

class Person:
    def __init__(self, name, surname, birthdate, address, telephone, email):
        self.name = name
        self.surname = surname
        self.birthdate = birthdate

        self.address = address
        self.telephone = telephone
        self.email = email

    def __str__(self):
        return "%s %s, born %s\nAddress: %s\nTelephone: %s\nEmail:%s" % (self.name, self.surname, self.birthdate, self.address, self.telephone, self.email)

jane = Person(
    "Jane",
    "Doe",
    datetime.date(1992, 3, 12), # year, month, day
    "No. 12 Short Street, Greenville",
    "555 456 0987",
    "jane.doe@example.com"
)

print(jane)

Jane Doe, born 1992-03-12
Address: No. 12 Short Street, Greenville
Telephone: 555 456 0987
Email:jane.doe@example.com


Note that when we insert the birthdate object into the output string with %s it will itself be converted to a string, so we don’t need to do it ourselves (unless we want to change the format).

It is also often useful to overload the comparison methods, so that we can use comparison operators on our person objects. By default, our person objects are only equal if they are the same object, and you can’t test whether one person object is greater than another because person objects have no default order.

Suppose that we want our person objects to be equal if all their attributes have the same values, and we want to be able to order them alphabetically by surname and then by first name. All of the magic comparison methods are independent of each other, so we will need to overload all of them if we want all of them to work – but fortunately once we have defined equality and one of the basic order methods the rest are easy to do. Each of these methods takes two parameters – self for the current object, and other for the other object:

In [18]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def __eq__(self, other): # does self == other?
        return self.name == other.name and self.surname == other.surname

    def __gt__(self, other): # is self > other?
        if self.surname == other.surname:
            return self.name > other.name
        return self.surname > other.surname

    # now we can define all the other methods in terms of the first two

    def __ne__(self, other): # does self != other?
        return not self == other # this calls self.__eq__(other)

    def __le__(self, other): # is self <= other?
        return not self > other # this calls self.__gt__(other)

    def __lt__(self, other): # is self < other?
        return not (self > other or self == other)

    def __ge__(self, other): # is self >= other?
        return not self < other

Note that other is not guaranteed to be another person object, and we haven’t put in any checks to make sure that it is. Our method will crash if the other object doesn’t have a name or surname attribute, but if they are present the comparison will work. Whether that makes sense or not is something that we will need to think about if we create similar types of objects.

Sometimes it makes sense to exit with an error if the other object is not of the same type as our object, but sometimes we can compare two compatible objects even if they are not of the same type. For example, it makes sense to compare 1 and 2.5 because they are both numbers, even though one is an integer and the other is a float.